In [1]:
!pip uninstall scikit-learn --yes
!pip uninstall imblearn --yes
!pip install scikit-learn==1.2.2
!pip install imblearn

Found existing installation: scikit-learn 1.2.2
Uninstalling scikit-learn-1.2.2:
  Successfully uninstalled scikit-learn-1.2.2
Found existing installation: imblearn 0.0
Uninstalling imblearn-0.0:
  Successfully uninstalled imblearn-0.0
Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for scikit-learn==1.2.2 from https://files.pythonhosted.org/packages/db/98/169b46a84b48f92df2b5e163fce75d471f4df933f8b3d925a61133210776/scikit_learn-1.2.2-cp311-cp311-win_amd64.whl.metadata
  Using cached scikit_learn-1.2.2-cp311-cp311-win_amd64.whl.metadata (11 kB)
Using cached scikit_learn-1.2.2-cp311-cp311-win_amd64.whl (8.3 MB)
Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for imblearn from https://files.pythonhosted.org/packages/81/a7/4179e6ebfd654bd0eac0b9c06125b8b4c96a9d0a8ff9e9507eb2a26d2d7e/imblearn-0.0-py2.py3-none-any.whl.metadata
  Using cached imblearn-0.0-py2.py3-

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#SK-Learn
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder,OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
import time
from sklearn.feature_selection import RFE, f_regression
from pprint import pprint
from sklearn.model_selection import GridSearchCV,cross_val_score,train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,recall_score,precision_score,f1_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from imblearn.over_sampling import SMOTE

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [3]:
df=pd.read_csv('Train_Dataset_(12.csv')

In [4]:
test_data = pd.read_csv('Test_Dataset_(1)_(1).csv')

In [5]:
df.head()

,EmployeeID,Age,TravelProfile,Department,HomeToWork,EducationField,Gender,HourlnWeek,Involvement,WorkLifeBalance,...,ESOPs,NumCompaniesWorked,OverTime,SalaryHikelastYear,WorkExperience,LastPromotion,CurrentProfile,MaritalStatus,MonthlyIncome,Attrition
0,5110001,35.0,Rarely,Analytics,5.0,CA,Male,69.0,1,1,...,1,1,1,20.0,7.0,2.0,NaN,M,18932.0,1
1,5110002,32.0,Yes,Sales,5.0,Statistics,Female,62.0,4,3,...,0,8,0,20.0,4.0,1.0,NaN,Single,18785.0,1
2,5110003,31.0,Rarely,Analytics,5.0,Statistics,F,45.0,5,3,...,1,3,0,26.0,12.0,1.0,3.0,Single,22091.0,0
3,5110004,34.0,Yes,Sales,10.0,Statistics,Female,32.0,3,2,...,1,1,0,23.0,5.0,1.0,3.0,Divorsed,20302.0,1
4,5110005,37.0,No,Analytics,27.0,Statistics,Female,49.0,3,4,...,1,8,0,21.0,12.0,1.0,9.0,Divorsed,21674.0,0


In [6]:
df.shape

(5180, 22)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5180 entries, 0 to 5179
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   EmployeeID          5180 non-null   int64  
 1   Age                 4864 non-null   float64
 2   TravelProfile       5180 non-null   object 
 3   Department          5056 non-null   object 
 4   HomeToWork          4925 non-null   float64
 5   EducationField      5180 non-null   object 
 6   Gender              5134 non-null   object 
 7   HourlnWeek          4893 non-null   float64
 8   Involvement         5180 non-null   int64  
 9   WorkLifeBalance     5180 non-null   int64  
 10  Designation         5142 non-null   object 
 11  JobSatisfaction     5180 non-null   int64  
 12  ESOPs               5180 non-null   int64  
 13  NumCompaniesWorked  5180 non-null   int64  
 14  OverTime            5180 non-null   int64  
 15  SalaryHikelastYear  5011 non-null   float64
 16  WorkEx

In [8]:

x_df = df.drop(columns = ['EmployeeID','Attrition'])

In [9]:
x_df

,Age,TravelProfile,Department,HomeToWork,EducationField,Gender,HourlnWeek,Involvement,WorkLifeBalance,Designation,JobSatisfaction,ESOPs,NumCompaniesWorked,OverTime,SalaryHikelastYear,WorkExperience,LastPromotion,CurrentProfile,MaritalStatus,MonthlyIncome
0,35.0,Rarely,Analytics,5.0,CA,Male,69.0,1,1,Executive,1,1,1,1,20.0,7.0,2.0,NaN,M,18932.0
1,32.0,Yes,Sales,5.0,Statistics,Female,62.0,4,3,Executive,2,0,8,0,20.0,4.0,1.0,NaN,Single,18785.0
2,31.0,Rarely,Analytics,5.0,Statistics,F,45.0,5,3,Manager,2,1,3,0,26.0,12.0,1.0,3.0,Single,22091.0
3,34.0,Yes,Sales,10.0,Statistics,Female,32.0,3,2,Manager,4,1,1,0,23.0,5.0,1.0,3.0,Divorsed,20302.0
4,37.0,No,Analytics,27.0,Statistics,Female,49.0,3,4,Manager,4,1,8,0,21.0,12.0,1.0,9.0,Divorsed,21674.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5175,36.0,Rarely,Analytics,13.0,CA,F,53.0,3,3,Manager,3,0,4,1,22.0,11.0,1.0,5.0,Single,22142.0
5176,NaN,Rarely,Marketing,9.0,CA,Female,66.0,1,4,Executive,4,1,9,0,23.0,8.0,0.0,2.0,Single,17109.0
5177,29.0,Rarely,Analytics,NaN,CA,Female,62.0,4,2,Executive,1,1,6,0,19.0,8.0,0.0,3.0,M,17532.0
5178,26.0,Rarely,Marketing,26.0,Statistics,Female,61.0,3,4,Executive,3,0,1,0,28.0,8.0,5.0,7.0,Divorsed,17942.0


In [10]:
x_df.isna().sum()

Age                   316
TravelProfile           0
Department            124
HomeToWork            255
EducationField          0
Gender                 46
HourlnWeek            287
Involvement             0
WorkLifeBalance         0
Designation            38
JobSatisfaction         0
ESOPs                   0
NumCompaniesWorked      0
OverTime                0
SalaryHikelastYear    169
WorkExperience        187
LastPromotion          70
CurrentProfile        311
MaritalStatus           0
MonthlyIncome          93
dtype: int64

In [11]:
x_df.isnull().sum()

Age                   316
TravelProfile           0
Department            124
HomeToWork            255
EducationField          0
Gender                 46
HourlnWeek            287
Involvement             0
WorkLifeBalance         0
Designation            38
JobSatisfaction         0
ESOPs                   0
NumCompaniesWorked      0
OverTime                0
SalaryHikelastYear    169
WorkExperience        187
LastPromotion          70
CurrentProfile        311
MaritalStatus           0
MonthlyIncome          93
dtype: int64

In [12]:
binary_num_cols = ['ESOPs','OverTime']
ordinal_num_cols = ['Involvement', 'WorkLifeBalance', 'JobSatisfaction']

In [13]:
x= df.drop(columns=['Attrition', 'EmployeeID'])
y = df['Attrition']

In [14]:
for col in ordinal_num_cols:
    x[col] = x[col].astype('category')
    test_data[col] = test_data[col].astype('category')

In [15]:
test_data

,EmployeeID,Age,TravelProfile,Department,HomeToWork,EducationField,Gender,HourlnWeek,Involvement,WorkLifeBalance,...,JobSatisfaction,ESOPs,NumCompaniesWorked,OverTime,SalaryHikelastYear,WorkExperience,LastPromotion,CurrentProfile,MaritalStatus,MonthlyIncome
0,6110001,18.0,No,NaN,9.0,CA,Male,80.0,3,2,...,3,1,1,0,17.0,0.0,0.0,0.0,Single,16904.0
1,6110002,20.0,Rarely,Analytics,28.0,Statistics,Female,59.0,1,3,...,1,1,2,1,18.0,2.0,1.0,NaN,Single,18994.0
2,6110003,50.0,Rarely,Analytics,19.0,CA,Female,76.0,3,3,...,5,0,5,1,22.0,18.0,3.0,3.0,M,18587.0
3,6110004,32.0,Rarely,Sales,23.0,Statistics,Female,73.0,5,2,...,3,0,4,1,17.0,5.0,3.0,3.0,Married,20559.0
4,6110005,39.0,Rarely,Analytics,7.0,CA,Male,42.0,4,1,...,5,1,4,0,20.0,9.0,1.0,7.0,Married,24991.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2625,6112626,46.0,Rarely,Sales,12.0,Marketing Diploma,Male,76.0,3,5,...,5,1,5,1,18.0,10.0,1.0,3.0,Married,26761.0
2626,6112627,29.0,Rarely,Analytics,22.0,CA,Male,80.0,4,4,...,5,0,2,0,18.0,7.0,4.0,4.0,Divorsed,19196.0
2627,6112628,44.0,Rarely,Analytics,8.0,CA,F,42.0,4,1,...,1,1,3,0,19.0,24.0,5.0,17.0,Married,25248.0
2628,6112629,NaN,Rarely,Analytics,11.0,Statistics,Female,NaN,4,3,...,4,0,2,0,18.0,2.0,1.0,1.0,Single,17261.0


In [16]:
for col in binary_num_cols:
    x[col] = x[col].astype('object')
    test_data[col] = test_data[col].astype('object')

In [17]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5180 entries, 0 to 5179
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   Age                 4864 non-null   float64 
 1   TravelProfile       5180 non-null   object  
 2   Department          5056 non-null   object  
 3   HomeToWork          4925 non-null   float64 
 4   EducationField      5180 non-null   object  
 5   Gender              5134 non-null   object  
 6   HourlnWeek          4893 non-null   float64 
 7   Involvement         5180 non-null   category
 8   WorkLifeBalance     5180 non-null   category
 9   Designation         5142 non-null   object  
 10  JobSatisfaction     5180 non-null   category
 11  ESOPs               5180 non-null   object  
 12  NumCompaniesWorked  5180 non-null   int64   
 13  OverTime            5180 non-null   object  
 14  SalaryHikelastYear  5011 non-null   float64 
 15  WorkExperience      4993 non-null   fl

In [18]:
cat_cols = [col for col in x.select_dtypes(include=['object']).columns.tolist() if col not in ordinal_num_cols]
num_cols = x.select_dtypes(exclude=['object', 'category']).columns.tolist()

In [19]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

In [20]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='N.A')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [21]:
ordinal_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ordinal', OrdinalEncoder())])

In [26]:
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('classifier', RandomForestClassifier(random_state=42,))])
# model = Pipeline(steps=[('preprocessor', preprocessor),
#                         ('classifier', GaussianNB())])

NameError: name 'preprocessor' is not defined

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
param_grid = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [None, 10, 20, 30],
    'classifier__criterion' : ["gini", "entropy", "log_loss"],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4]
}

In [ ]:
grid_search = GridSearchCV(model, param_grid, cv=5, n_jobs=-1, scoring='accuracy')
grid_search.fit(X_train, y_train)

In [ ]:
print("Best parameters found: ", grid_search.best_params_)
best_model = grid_search.best_estimator_

In [ ]:
y_pred = best_model.predict(X_val)
print('Validation Accuracy:', accuracy_score(y_val, y_pred))#0.9884169884169884

In [ ]:
X_test = test_data.drop(columns=['EmployeeID'])
test_data['Attrition'] = best_model.predict(X_test)

In [ ]:
submission = test_data[['EmployeeID', 'Attrition']]
submission.to_csv('submission.csv', index=False)
print("Submission file created.")

In [ ]:
from google.colab import files
files.download('submission.csv')